In [2]:
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

In [3]:
#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file('test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

4893335/4893335 [==============================] - 0s 0us/step


In [4]:
#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
text_train = open(train_file,'rb').read().decode(encoding='utf-8')
text_test = open(test_file,'rb').read().decode(encoding='utf-8')

In [5]:
print(text_train)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [6]:
# train y값 label을 먼저 뽑는다. 
a=text_train[:500].split('\n')[1:]
#줄마다 나눠서 a에 넣는다.
print(a[:5])
print(a[3].count('\t'))
# a[5]


['9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0', '3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1', '10265843\t너무재밓었다그래서보는것을추천한다\t0', '9045019\t교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정\t0', '6483659\t사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다\t1']
2


In [7]:
# '9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0'
# '9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'
a[0].split('\t')[2]  # 감상평의 라벨 0= 부정,  1= 긍정

'0'

In [ ]:
# for row in text_train.split('\n')[1:]: 
#     print(row) #여기 한줄 뽑아서 
#     print(row.split('\t')[2]) #\t 로 나눠서 2번째 원소인 라벨을 뽑는다
#     print( int(row.split('\t')[2]) ) # int() 해서 정수형으로 바꿔준다.

In [9]:
# 데이터 정제 

print(text_test[:200])

#각 줄을 \n으로 분리하겠다.
#헤더를 제외하겠다. 

#한줄마다 \t로 분할해서 문장과 점수로 나눈다
#감상평은 \t로 구분되어있음
# [1:]으로 헤더제거

#감상평의 평점만 뽑아서 정수화 행렬로 만들어줬다.
y_train= np.array([ [int(row.split('\t')[2])] for row in text_train.split('\n')[1:] if row.count('\t')>0])
y_test= np.array([ [int(row.split('\t')[2])] for row in text_test.split('\n')[1:] if row.count('\t')>0])
print(y_train.shape) # (150000개의 평점 , 1개의 열)


id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심
(150000, 1)


In [ ]:
text_train.split('\n')  #한줄씩 분리하고
text_train.split('\n')[1:]  #헤더 처리
#row.count('\t')>0   평점이 존재하면
# row.split('\t')[1]  감상평만 추출

In [12]:
#X 값 추출 = 리뷰문장
x_text_train = [ row.split('\t')[1] for row in text_train.split('\n')[1:] if row .count('\t')>0  ]
x_text_train[:3]


['아 더빙.. 진짜 짜증나네요 목소리',
 '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다']

In [ ]:
# !pip install konlpy

In [ ]:
#위에서 추출한 문장을 토큰화해볼것이다. => OKT 방법을 사용할것이다.
from konlpy.tag import Okt
#konlpy 패키지에있는 tag 모듈에 Okt 메서드를 불러온다.

x_train_tokenized = []
for i in x_text_train:
    x_train_tokenized.append( Okt().morphs( i ) ) 

#시간이 오래걸리네 #GPU로 5분정도
x_train_tokenized


In [26]:
# 토큰화된 학습 데이터
# x_train_tokenized[:100]
# 나눠진 토큰들을 임베딩할것이다. 
len(x_train_tokenized)

total_len = 0
for i in x_train_tokenized:
    total_len += len(i)
print(total_len)  # 토큰 2,156,163 개  => 은, 는, 이, 가 이게 많이 나오면 중요 키워드인가? 
# 그렇지않다. => 그럼 제거한다


2156163


만약 의문문 대해서 커스텀을하고싶으면 


In [20]:
# Okt().morphs( '오늘 뭐먹었어?' )
# Okt().morphs( '오늘 뭐했어?' ) #의문문에 대한 구분이 불가능 해보인다. 

['오늘', '뭐', '했어', '?']

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words= 20000 ) #빈도가 높은것 순서대로 2만개만 다시 토큰화한다.
#빈도수가 2만개 보다 낮은것은 그냥 0으로 처리한다.

tokenizer.fit_on_texts( x_train_tokenized )
 

# 토큰(texts:형태소)들을 시계열데이터로 변환한다. == 이 과정을 정수 인코딩딩이라한다.  
sequences = tokenizer.texts_to_sequences( x_train_tokenized )

#이렇게 숫자로 바뀌면 이제 컴퓨터가 처리할수있는 데이터가 된다.
sequences


In [ ]:
# 패딩
from tensorflow.keras.preprocessing import pad_sequences
X_train = pad_sequences(sequences,padding='post')  #학습데이터 완성
y_train

# [1] => 세개 공간을 만들고 싶다. = 앞에[0 0 1], post뒤에[1 0 0]

In [32]:
# 딥러닝 모델
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

#딥러닝기반 언어(텍스트정제 => 숫자로된 시계열데이터로 들어온다 )처리모델 
model = Sequential([

])

